In [6]:
import cv2
import numpy as np

In [7]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)
(x,y,w,h) = tuple(face_rects[0])
track_window = (x,y,w,h)
roi = frame[y:y+h,x:x+h]
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])
roi_hist = cv2.normalize(roi_hist,None,0,255,cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.5)

while True:
    ret, frame = cap.read()
    if ret:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        back_projection = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        ret, track_window = cv2.meanShift(back_projection,track_window,term_crit)
        (x,y,w,h) = track_window
        img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),6)
        cv2.imshow('MeanShift Tracking',img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

In [9]:
cap = cv2.VideoCapture(0)
ret,frame = cap.read()
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)
(face_x,face_y,w,h) = tuple(face_rects[0])
track_window = (face_x,face_y,w,h)
roi = frame[face_y:face_y+h,face_x:face_x+w]
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])
roi_norm = cv2.normalize(roi_hist,None,0,255,cv2.NORM_MINMAX)
term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_norm,[0,180],1)
        ret, track_window = cv2.CamShift(dst,track_window,term_criteria)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img = cv2.polylines(frame,[pts],True,(0,255,0),4)
        cv2.imshow('CamShift Object Tracking',img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()